# Entrenamiento de un Modelo de spaCy para Detección de Entidades Nombradas (NER)
---
Este notebook tiene como objetivo entrenar un modelo de procesamiento de lenguaje natural (NLP) utilizando la biblioteca spaCy para realizar tareas de detección de entidades nombradas (NER, por sus siglas en inglés).

La detección de entidades nombradas es una técnica clave en NLP que permite identificar y clasificar entidades específicas en un texto, como nombres de personas, organizaciones, ubicaciones, fechas, cantidades, entre otras.

## Objetivos del Notebook

1. **Preparar los datos de entrenamiento**: Generar un conjunto de datos etiquetados para el entrenamiento del modelo.
2. **Configurar el modelo**: Definir los parámetros del modelo y configurar spaCy para trabajar con datos personalizados.
3. **Entrenar el modelo**: Utilizar los datos para ajustar el modelo de spaCy y mejorar su capacidad para detectar las entidades deseadas.
4. **Evaluar el modelo**: Validar el rendimiento del modelo utilizando métricas como precisión, exhaustividad y F1.
5. **Guardar y reutilizar el modelo**: Exportar el modelo entrenado para integrarlo en otras aplicaciones.



---

In [30]:
import pandas as pd
import random

import pandas as pd
from sklearn.model_selection import train_test_split
import spacy

import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example
from pathlib import Path
from ast import literal_eval
from spacy.tokens import Doc, Span, Token
from spacy.scorer import Scorer

### 1.1 Obtener ejemplos de calles

Para crear nuestro set de entrenamiento, utilizaremos las calles de la ciudad de Madrid. Así, crearemos ejemplos con nombres reales que más tarde ayudarán a generalizar al modelo.

Descargamos los datos del callejero de Madrid desde [aquí](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=b3c41f3cf6a6c410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)


In [3]:
df_callejero = pd.read_csv('../data/VialesVigentes_20241226.csv', sep = ';', encoding = 'cp1252')

Una vez cargados los datos, los visualizamos para entender con qué información estamos trabajando. Nos encontramos con 15 columnas, de las cuales solo nos interesan 3: VIA_CLASE, VIA_PAR y VIA_NOMBRE.

In [4]:
df_callejero.head(5)

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,COD_VIA_COMIENZA,CLASE_COMIENZA,PARTICULA_COMIENZA,NOMBRE_COMIENZA,NOMBRE_ACENTOS_COMIENZA,COD_VIA_TERMINA,CLASE_TERMINA,PARTICULA_TERMINA,NOMBRE_TERMINA,NOMBRE_ACENTOS_TERMINA
0,31001337,AUTOVÍA,NaN,A-1,A-1,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
1,31001336,AUTOVÍA,NaN,A-2,A-2,310200,CALLE,DE,FRANCISCO SILVELA,FRANCISCO SILVELA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
2,31001342,AUTOVÍA,NaN,A-3,A-3,480800,PLAZA,DE,MARIANO DE CAVIA,MARIANO DE CAVIA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
3,31001334,AUTOVÍA,NaN,A-4,A-4,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
4,31001341,AUTOVÍA,NaN,A-42,A-42,468400,AVENIDA,DEL,MANZANARES,MANZANARES,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL


Una vez seleccionadas las columnas de interés, crearemos una columna adicional que sea la concatenación de la partícula y el nombre de la vía, ya que no estamos interesados de que nuestro modelo detecte las partículas de forma distinta al nombre de la vía.

Así mismo, eliminaremos los registros que contengan vacíos.

In [5]:
df_calles = df_callejero[df_callejero.VIA_CLASE != 'AUTOVÍA'][['VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE']]

df_calles['NOMBRE_VIA'] = df_calles['VIA_PAR'] + ' ' + df_calles['VIA_NOMBRE']
df_calles.drop(columns = ['VIA_PAR', 'VIA_NOMBRE'], inplace  = True)


df_calles.rename(columns={'VIA_CLASE': 'TIPO_VIA'}, inplace = True)
df_calles.dropna(inplace =  True)

In [6]:
df_calles.head(5)

,TIPO_VIA,NOMBRE_VIA
7,CALLE,DEL ABAD JUAN CATALAN
8,CALLE,DE LA ABADA
9,CALLE,DE LOS ABADES
10,CALLE,DE LA ABADESA
11,CALLE,DE ABALOS


### 1.2. Generar direcciones ficticias

Para terminar de crear las direcciones ficticias, añadiremos más señas a las calles de Madrid. Para ello, de manera aleatoria, incluiremos información sobre el número y la combinación piso-letra

In [25]:
def generar_datos_entrenamiento(df, n):
    """
    Genera N líneas aleatorias a partir de un DataFrame base.
    
    Args:
        df (pd.DataFrame): DataFrame base para seleccionar líneas.
        n (int): Número de líneas aleatorias a generar.
    
    Returns:
        pd.DataFrame: DataFrame con N líneas aleatorias generadas.
    """
    lineas = []
    for _ in range(n):
        # Seleccionar una línea aleatoria del DataFrame base
        row = df.sample(1).copy()

        # Agregar un número aleatorio
        pre_numero = random.choice(['n*', 'nr', 'nº', '', 'n'])
        numero = random.randint(1, 50)
        post_numero = random.choice(['', '', '', ','])
        row['NUMERO'] = f'{pre_numero}{numero}{post_numero}'

        # Agregar combinación aleatoria de piso y letra
        piso = random.randint(1, 10)
        letra = random.choice(['A', 'B', 'C', 'D', 'IZDA', 'DCHA', 'CTRO'])
        espacio = random.choice([' ', '*', 'º', '', '-'])
        combinacion = f"{piso}{espacio}{letra}"
        row['RESTO'] = combinacion

        # Agregar la línea generada a la lista
        lineas.append(row)

    # Combinar todas las líneas en un nuevo DataFrame
    return pd.concat(lineas, ignore_index=True)


In [48]:
train_set = generar_datos_entrenamiento(df_calles, 7000)

### 1.3. Estructurar los datos para el modelo

Una vez terminamos de generar las direcciones ficticias, daremos forma a nuestros datos para poder entrenar el modelo de spacy. Esta estructura es específica para la librería SpaCy.

In [49]:
def crear_entidades(row):
    texto = row['texto']
    entidades = []
    start = 0
    for col, label in zip(['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO'], ['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO']):
        value = str(row[col])
        start = texto.find(value, start)  # Buscar el índice inicial de la entidad
        if start != -1:
            end = start + len(value)  # Índice final de la entidad
            entidades.append((start, end, label))
            start = end  # Actualizar el inicio para evitar errores en textos repetidos
    return (texto, {'entities': entidades})

In [50]:
train_set['texto'] = train_set['TIPO_VIA'] + ' ' + train_set['NOMBRE_VIA'] + ' ' + train_set['NUMERO'].astype(str) + ' ' + train_set['RESTO']
train_set['entidades'] = train_set.apply(crear_entidades, axis=1)

lista = train_set['entidades'].to_list()

train, test = train_test_split(lista, test_size=0.25, random_state=8)

In [51]:
lista[0]

('CALLE DE LOS MARTIRES DE PARACUELLOS n44 3 D',
 {'entities': [(0, 5, 'TIPO_VIA'),
   (6, 36, 'NOMBRE_VIA'),
   (37, 40, 'NUMERO'),
   (41, 44, 'RESTO')]})

## 2. Configurar el modelo

Para entrenar el modelo, primeramente necesitamos seleccionar qué componentes del pipeline del modelo pre-configurado queremos modificar con nuestros datos de entrenamiento.

In [52]:
nlp = spacy.load('en_core_web_sm')

ner = nlp.get_pipe('ner')

for _,annotations in train:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

pipe_exceptions = ['ner', 'trf_wordpiecer', 'trf_tok2vec']
unaffected_pipe = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

## 3. Entrenar el modelo

In [ ]:
nr_iter = 50

with nlp.disable_pipes(*unaffected_pipe):
    for iteration in range(nr_iter):
        random.shuffle(train)
        losses = {}

        batches = minibatch(train, size=8)
        for batch in batches:
            example = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example.append(Example.from_dict(doc, annotations))

            nlp.update(example, drop = 0.3, losses=losses)

        print(f"Iteration {iteration} - Losses: {losses}")


## 4. Evaluar el modelo



In [47]:
# Test the model
test_text = "PLAZA ELÍPTICA 8, 3*B"
doc = nlp(test_text)

print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('PLAZA', 'TIPO_VIA'), ('DE ELÍPTICA', 'NOMBRE_VIA'), ('8,', 'NUMERO'), ('3*B', 'RESTO')]


In [31]:
examples = []
scorer = Scorer()
for text, annotations in test:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    example.predicted = nlp(str(example.predicted))
    examples.append(example)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.9966711051930759,
 'ents_r': 0.998,
 'ents_f': 0.9973351099267155,
 'ents_per_type': {'TIPO_VIA': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'NOMBRE_VIA': {'p': 0.9920212765957447,
   'r': 0.9946666666666667,
   'f': 0.9933422103861518},
  'NUMERO': {'p': 0.9973404255319149, 'r': 1.0, 'f': 0.9986684420772304},
  'RESTO': {'p': 0.9973333333333333,
   'r': 0.9973333333333333,
   'f': 0.9973333333333333}},
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_typ

In [34]:
results_scorer = scorer.score(examples)
results_scorer['ents_per_type']

{'TIPO_VIA': {'p': 1.0, 'r': 1.0, 'f': 1.0},
 'NOMBRE_VIA': {'p': 0.9920212765957447,
  'r': 0.9946666666666667,
  'f': 0.9933422103861518},
 'NUMERO': {'p': 0.9973404255319149, 'r': 1.0, 'f': 0.9986684420772304},
 'RESTO': {'p': 0.9973333333333333,
  'r': 0.9973333333333333,
  'f': 0.9973333333333333}}